In [30]:
from parameter_search import util
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame
import csv
import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.models import Phrases
assert gensim.models.doc2vec.FAST_VERSION == 1, "this will be painfully slow otherwise"
import nltk, re, random
import datetime
import multiprocessing
from ast import literal_eval
import pickle
from tqdm import tqdm
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn import linear_model
import datetime
from scipy.cluster.hierarchy import dendrogram, linkage
#%matplotlib inline

In [2]:
data_path = "E:/dataset/Amazon/"
save_path = "E:/dataset/MasterThesis/FINAL/preprocess_data/"
model_path = "E:/dataset/MasterThesis/FINAL/doc2vec/"
category_list = ["Electronics","Beauty","Clothing_Shoes_and_Jewelry"]

In [32]:
%%time
documents = util.load_data(save_path, category_list, tagby=True) 

Electronics category is finished
Beauty category is finished
Clothing_Shoes_and_Jewelry category is finished
Wall time: 4min 27s


In [33]:
len(documents)

300000

In [34]:
#parameters
y = np.array([1] * 100000 + [0] * 100000 + [-1] * 100000)
tags = [doc.tags[0] for doc in documents]
tag_dict = {}
for index, tag in enumerate(tags):
    tag_dict[tag] = y[index]
    
window = [2,4,6,8,10]
size = [50,100,200,300,400,500]
alpha, min_alpha, passes = (0.02, 0.001, 10)
alpha_delta = (alpha-min_alpha)/passes

df_param = pd.DataFrame(columns=[['epoch', 'training_time', 'modeling_time', 'dimension', 'window', 'avg_accuracy']])

print("start : ", datetime.datetime.now())
for s in size:
    for w in window:
        #PV_DM w/average
        model = Doc2Vec(dm=1, dm_mean=1, size=s, min_count=50, window=w, workers=12, 
                        alpha=alpha, min_alpha=min_alpha)
        model.build_vocab(documents)

        for epoch in range(passes):
            print("num of epoch %s "%(epoch+1))
            start = time.time()
            random.shuffle(documents)
            model.train(documents)
            model.alpha -= alpha_delta  # decrease the learning rate
            model.min_alpha = model.alpha  # fix the learning rate, no decay
            end = time.time()

            start2 = time.time()
            X = np.array([model.docvecs[tag] for tag in tags])
            y = [tag_dict[tag] for tag in tags]
            y = np.array(y)
            
            accuracy = []
            kf = KFold(X.shape[0], n_folds=10, shuffle=True, random_state=10)
            for k, (train_index, test_index) in enumerate(kf):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                sgdreg = linear_model.SGDClassifier(loss='log', penalty='l2', n_jobs=-1, alpha=0.0001, n_iter=5, random_state =111)
                sgdreg.fit(X_train, y_train)
                accuracy.append(np.mean(sgdreg.predict(X_test) == y_test))
            end2 = time.time()
            print("epoch : %i, training time: %f, modeling time: %f, dimension size : %i, window size : %i, avg accuracy : %f" %(epoch+1, end-start, end2-start2, s, w, np.mean(accuracy)))
            df_param.loc[len(df_param)] = [epoch+1, start-end, start2-end2, s, w, np.mean(accuracy)]
            #model.save(model_path + "model_" + str(w) + '_' + str(s) + '_' + str(epoch+1))
print("end : ", datetime.datetime.now())

start :  2016-12-12 02:19:23.873986
num of epoch 1 
epoch : 1, training time: 94.910786, modeling time: 7.684298, dimension size : 50, window size : 2, avg accuracy : 0.805577
num of epoch 2 
epoch : 2, training time: 94.926707, modeling time: 7.632461, dimension size : 50, window size : 2, avg accuracy : 0.846273
num of epoch 3 
epoch : 3, training time: 95.410011, modeling time: 7.764786, dimension size : 50, window size : 2, avg accuracy : 0.859070
num of epoch 4 
epoch : 4, training time: 95.008087, modeling time: 7.397732, dimension size : 50, window size : 2, avg accuracy : 0.864263
num of epoch 5 
epoch : 5, training time: 95.168256, modeling time: 7.753154, dimension size : 50, window size : 2, avg accuracy : 0.866477
num of epoch 6 
epoch : 6, training time: 95.471654, modeling time: 7.480951, dimension size : 50, window size : 2, avg accuracy : 0.867317
num of epoch 7 
epoch : 7, training time: 95.164119, modeling time: 7.908519, dimension size : 50, window size : 2, avg accu

In [36]:
df_param.to_csv('D:/Dropbox/2016-2/졸업논문/FINAL_result/ParamSearch/result.csv',index=False)